In [4]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%autoreload 2
from collections import defaultdict
from datetime import datetime
import h5py
import matplotlib.pyplot as plt
from neural_analysis.matIO import loadmat
import numpy as np
import os
import pandas as pd
from scipy.signal import butter, filtfilt, freqz
import sys
import time
from tqdm.auto import tqdm

sys.path.append('../..')
from ld_utils import compute_eigs, get_data_sigma_norms
from up_down import get_up_down
from utils import get_binary_stimuli, get_sample_interval, load, save, to_string

## Load Neural Data

In [6]:
# filename = '../../__data__/Mary-Anesthesia-20160809-01.mat'
filename = r'/home/adameisen/millerdata/common/datasets/anesthesia/mat/propofolPuffTone/Mary-Anesthesia-20160809-01.mat'
# filename = r'/home/adameisen/millerdata/common/datasets/anesthesia/mat/propofolPuffTone/MrJones-Anesthesia-20160121-01.mat'
# filename = r'/home/adameisen/common/datasets/anesthesia/mat/propofolWakeUp/Mary-Anesthesia-20170203-02.mat'
print("Loading data ...")
start = time.process_time()
electrode_info, lfp, lfp_schema, session_info, spike_times, unit_info = loadmat(filename, variables=['electrodeInfo', 'lfp', 'lfpSchema', 'sessionInfo', 'spikeTimes', 'unitInfo'], verbose=False)
spike_times = spike_times[0]
dt = lfp_schema['smpInterval'][0]
T = lfp.shape[0]

f = h5py.File(filename, 'r')
airPuff_binary, audio_binary = get_binary_stimuli(f)

print(f"Data loaded (took {time.process_time() - start:.2f} seconds)")

Loading data ...
Data loaded (took 78.40 seconds)


# VAR

In [8]:
data_path = "../../__data__/VAR"
os.makedirs(data_path, exist_ok=True)
# areas = np.unique(electrode_info['area'])
# areas = ['vlPFC']
# unit_indices = np.arange(lfp.shape[1])[pd.Series(electrode_info['area']).isin(areas)]
PCA_dim = -1
end_step = T - int(30/dt) # take off 30 seconds
# end_step = 5000
# for window in [2.5]:
for window in [0.15, 0.25, 0.5, 1, 2.5, 5]:
    stride = window
    results = {}
    
    for area in np.unique(electrode_info['area']):
        areas = [area]
        unit_indices = np.arange(lfp.shape[1])[pd.Series(electrode_info['area']).isin(areas)]

        data = lfp[:end_step, unit_indices]
        print(f"window = {window}, area = {area}")
        results[area] = compute_eigs(data, dt, window, stride, return_sigma_norms=True, PCA_dim=PCA_dim, verbose=True)
        results[area]['end_step'] = end_step
        results[area]['data_sigma_norms'] = get_data_sigma_norms(data, dt, window, stride)
        del data
    
    print(f"window = {window}, all areas")
    areas = np.unique(electrode_info['area'])
    unit_indices = np.arange(lfp.shape[1])[pd.Series(electrode_info['area']).isin(areas)]
    data = lfp[:end_step, unit_indices]
    print(f"window = {window}")
    results['all'] = compute_eigs(data, dt, window, stride, return_sigma_norms=True, PCA_dim=PCA_dim, verbose=True, return_sigma_normsvisual)
    results['all']['end_step'] = end_step
    results['all']['data_sigma_norms'] = get_data_sigma_norms(data, dt, window, stride)
    del data
    
    save(results, os.path.join(data_path, f"results_window_{window}_stride_{stride}_PCA_dim_{PCA_dim}"))

window = 0.15, area = 7b


  0%|          | 0/47035 [00:00<?, ?it/s]

window = 0.15, area = CPB


  0%|          | 0/47035 [00:00<?, ?it/s]

window = 0.15, area = FEF


...

  0%|          | 0/1411 [00:00<?, ?it/s]

window = 5, area = vlPFC


  0%|          | 0/1411 [00:00<?, ?it/s]

window = 5, all areas
window = 5


  0%|          | 0/1411 [00:00<?, ?it/s]